In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display as ipy_display


conn = sqlite3.connect("Chinook.sqlite")


query = """
SELECT 
    Customer.LastName, 
    Customer.FirstName, 
    Track.Name AS Name, 
    Album.Title AS Title
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
ORDER BY Customer.LastName ASC, Customer.FirstName ASC;
"""


df = pd.read_sql_query(query, conn)


conn.close()


def paginate_data(df, page_num, rows_per_page):
   
    start_idx = (page_num - 1) * rows_per_page
    end_idx = start_idx + rows_per_page
   
    ipy_display(df.iloc[start_idx:end_idx])


rows_per_page = 10
max_pages = len(df) // rows_per_page + (1 if len(df) % rows_per_page > 0 else 0)
page_num = 1


def on_prev_button_clicked(change):
    global page_num
    if page_num > 1:
        page_num -= 1
    update_display()

def on_next_button_clicked(change):
    global page_num
    if page_num < max_pages:
        page_num += 1
    update_display()


def update_display():
    print(f"\n\U0001F4CC Page {page_num} of the dataset:")
    paginate_data(df, page_num, rows_per_page)


prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")


prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)


widget_box = widgets.HBox([prev_button, next_button])


display(widget_box)
update_display()


print("\n\U0001F4CC Dataset Shape (Rows, Columns):", df.shape)


unique_customers = df[['LastName', 'FirstName']].drop_duplicates()
print("\n\U0001F4CC Unique Customers Count:", unique_customers.shape[0])


customer_purchases = df.groupby(['LastName', 'FirstName'], as_index=False).agg(Total_Purchases=('Name', 'count'))


print("\n\U0001F4CC Top 10 Customers by Number of Purchases:")
display(customer_purchases.sort_values(by='Total_Purchases', ascending=False).head(10))


df.to_csv("customer_purchases.csv", index=False)
print("\n✅ Data exported to 'customer_purchases.csv'")




📌 Page 1 of the dataset:


,LastName,FirstName,Name,Title
0,Almeida,Roberto,Right Next Door to Hell,Use Your Illusion I
1,Almeida,Roberto,In The Evening,In Through The Out Door
2,Almeida,Roberto,Fool In The Rain,In Through The Out Door
3,Almeida,Roberto,Saudade Dos Aviões Da Panair (Conversando No Bar),Minas
4,Almeida,Roberto,Caso Você Queira Saber,Minas
5,Almeida,Roberto,Dance,Ace Of Spades
6,Almeida,Roberto,Aos Leões,Demorou...
7,Almeida,Roberto,Demorou!,Demorou...
8,Almeida,Roberto,Home Sweet Home,Motley Crue Greatest Hits
9,Almeida,Roberto,Intro,From The Muddy Banks Of The Wishkah [Live]



📌 Dataset Shape (Rows, Columns): (2240, 4)

📌 Unique Customers Count: 59

📌 Top 10 Customers by Number of Purchases:


,LastName,FirstName,Total_Purchases
0,Almeida,Roberto,38
43,Rocha,Alexandre,38
31,Miller,Dan,38
32,Mitchell,Aaron,38
33,Murray,Steve,38
34,Muñoz,Enrique,38
35,Nielsen,Kara,38
36,O'Reilly,Hugh,38
37,Pareek,Manoj,38
38,Peeters,Daan,38



✅ Data exported to 'customer_purchases.csv'


In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display as ipy_display

# Connect to the Chinook SQLite database
conn = sqlite3.connect("Chinook.sqlite")

# Define the SQL query
query = """
SELECT 
    Customer.LastName, 
    Customer.FirstName, 
    Track.Name AS Name, 
    Album.Title AS Title
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
ORDER BY Customer.LastName ASC, Customer.FirstName ASC;
"""

# Load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Function to paginate through the DataFrame
def paginate_data(df, page_num, rows_per_page):
    # Calculate the start and end indices for the pagination
    start_idx = (page_num - 1) * rows_per_page
    end_idx = start_idx + rows_per_page
    # Display the specific page of the DataFrame
    ipy_display(df.iloc[start_idx:end_idx])

# Set up the number of rows per page and the current page number
rows_per_page = 10
max_pages = len(df) // rows_per_page + (1 if len(df) % rows_per_page > 0 else 0)
page_num = 1

# Create pagination buttons
def on_prev_button_clicked(change):
    global page_num
    if page_num > 1:
        page_num -= 1
    update_display()

def on_next_button_clicked(change):
    global page_num
    if page_num < max_pages:
        page_num += 1
    update_display()

# Function to update display based on current page
def update_display():
    print(f"\n\U0001F4CC Page {page_num} of the dataset:")
    paginate_data(df, page_num, rows_per_page)

# Create 'Previous' and 'Next' buttons
prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")

# Bind the buttons to the functions
prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)

# Display the buttons in a horizontal box layout
widget_box = widgets.HBox([prev_button, next_button])

# Display the buttons and initial page of data
display(widget_box)
update_display()

# Check data completeness
print("\n\U0001F4CC Dataset Shape (Rows, Columns):", df.shape)

# Count unique customers
unique_customers = df[['LastName', 'FirstName']].drop_duplicates()
print("\n\U0001F4CC Unique Customers Count:", unique_customers.shape[0])

# Group by customer and count purchases
customer_purchases = df.groupby(['LastName', 'FirstName'], as_index=False).agg(Total_Purchases=('Name', 'count'))

# Display top customers by purchase count
print("\n\U0001F4CC Top 10 Customers by Number of Purchases:")
display(customer_purchases.sort_values(by='Total_Purchases', ascending=False).head(10))

# Export data to CSV
df.to_csv("customer_purchases.csv", index=False)
print("\n✅ Data exported to 'customer_purchases.csv'")

